In [2]:
import geehydro
from rasterio.windows import from_bounds
import rasterio as rio
from rasterio.enums import Resampling
import os
from os.path import join, basename
import numpy as np
import pandas as pd
#import rioxarray
import matplotlib.pyplot as plt
import rasterio as rio
#from osgeo import gdal
from rasterio.windows import Window , from_bounds, crop
import folium
import pyproj
from glob import glob

In [3]:
home_dir = '/home/zacharykeskinen/Documents/uavsar/'
data_dir = join(home_dir,'data')
url_dir = join(data_dir, 'urls')
imgs_dir = join(home_dir, 'imgs')
auc_dir = join(data_dir, 'aucillary')
low_auc_dir = join(auc_dir, 'lowman')

In [52]:
loc_dir = (join(imgs_dir, os.listdir(imgs_dir)[0]))

class Image:
    def __init__(self, fp, dtype, pol, real_img):
        self.fp = fp
        self.dtype = dtype
        self.pol = pol
        self.comp = real_img
    array = None
    top = None
    bottom = None
    left = None
    right = None
    crs = None
    transform = None
    window = None
    nodata = None

img_list = []
        
for img in os.listdir(loc_dir):
    if 'csv' in img:
        desc = pd.read_csv(join(loc_dir, img))
    if 'tif' in img:
        if len(basename(img).split('.')) == 3:
            fp = join(loc_dir, img)
            real_img = 'real'
            dtype = basename(img).split('.')[-2]
            pol = np.nan
            img_list.append(Image(fp, dtype, pol, real_img))
        else:
            fp = join(loc_dir, img)
            real_img = basename(img).split('.')[-2]
            dtype = basename(img).split('.')[-3]
            pol = basename(img).split('.')[0].split('_')[-2][4:]
            img_list.append(Image(fp, dtype, pol, real_img))

In [56]:
with rio.open(img_list[5].fp) as src:
    target_shape = img.shape
    crs = src.crs
    transform_affine = src.transform
    boundary = src.bounds
    my_proj = pyproj.Proj(str(crs).lower())#your data crs
    left_bottom = my_proj(boundary[0],boundary[1], inverse=True)
    right_top = my_proj(boundary[2],boundary[3], inverse=True)
    print(left_bottom)
    print(right_top)
    
    img = src.read(1)
    print(np.isnan(img).sum()/np.prod(img.shape))
    print(np.nanmax(img))
    print(np.nanmin(img))
    print(np.nanmean(img))

(-116.33319779194856, 43.55349529058187)
(-114.80929800665056, 44.51150448542751)
0.0
29.709307
0.0
0.06938995


In [42]:
auc_imgs = glob(low_auc_dir + '/*.tif', recursive = True)

In [43]:
with rio.open(img_list[1].fp) as src:
    img = src.read(1)
    target_shape = img.shape

for im in auc_imgs:
    with rio.open(im) as src:
        boundary = src.bounds
        crs = src.crs
        transform_affine = src.transform
        my_proj = pyproj.Proj(str(crs).lower())#your data crs
        left = (my_proj(boundary[0],boundary[1], inverse=True))[0]
        bottom = (my_proj(boundary[0],boundary[1], inverse=True))[1]
        right = (my_proj(boundary[2],boundary[3], inverse=True))[0]
        top = (my_proj(boundary[2],boundary[3], inverse=True))[1]
        img = src.read(1)
        print(basename(im))
        print(img.shape)
        print(np.isnan(img).sum()/np.prod(img.shape))
        print(np.nanmax(img))
        print(np.nanmin(img))
        print(np.nanmean(img))
        
        if 'tree' in im:
            resamp_tech = Resampling.bilinear
        else:
            resamp_tech = Resampling.nearest
        print(resamp_tech)
        # resample data to target shape
        resam = src.read(
            out_shape=(
                src.count,
                target_shape[0],
                target_shape[1]
            ),
            resampling=resamp_tech
        )[0]

        # scale image transform
        transform_resamp = src.transform * src.transform.scale(
            (src.width / resam.shape[-1]),
            (src.height / resam.shape[-2])
        )
        
        print(resam.shape)
        print(np.isnan(resam).sum()/np.prod(resam.shape))
        print(np.nanmax(resam))
        print(np.nanmin(resam))
        print(np.nanmean(resam))
        
        fp_new = im.replace('.tif','_resamp.tif')
        print(fp_new)
        profile = src.profile
        profile.update(
            height = resam.shape[0],
            width = resam.shape[1],
            transform = transform_resamp)
        with rio.open(fp_new, 'w', **profile) as dst:
            dst.write(resam, 1)

land_class.tif
(10693, 16965)
0.0
126
20
88.03801382908888
Resampling.nearest
(21603, 23940)
0.0
126
20
88.03744724956398
/home/zacharykeskinen/Documents/uavsar/data/aucillary/lowman/land_class_resamp.tif
lithography.tif
(10693, 16965)
0.0
19
0
5.909522730260113
Resampling.nearest
(21603, 23940)
0.0
19
0
5.9095195343819436
/home/zacharykeskinen/Documents/uavsar/data/aucillary/lowman/lithography_resamp.tif
trees.tif
(10693, 16965)
0.0008662798067403723
76.0
0.0
23.067721677386118
Resampling.bilinear
(21603, 23940)
0.0009447580128552801
73.0
0.0
23.06897104988965
/home/zacharykeskinen/Documents/uavsar/data/aucillary/lowman/trees_resamp.tif
land_forms.tif
(10693, 16965)
0.0
42
11
27.749903847290796
Resampling.nearest
(21603, 23940)
0.0
42
11
27.749908381254173
/home/zacharykeskinen/Documents/uavsar/data/aucillary/lowman/land_forms_resamp.tif


In [44]:
auc_imgs = glob(low_auc_dir + '/*resamp.tif', recursive = True)

In [46]:
for im in auc_imgs:
    with rio.open(im) as src:
        boundary = src.bounds
        crs = src.crs
        transform_affine = src.transform
        my_proj = pyproj.Proj(str(crs).lower())#your data crs
        left = (my_proj(boundary[0],boundary[1], inverse=True))[0]
        bottom = (my_proj(boundary[0],boundary[1], inverse=True))[1]
        right = (my_proj(boundary[2],boundary[3], inverse=True))[0]
        top = (my_proj(boundary[2],boundary[3], inverse=True))[1]
        img = src.read(1)
        print(basename(im))
        print(src.bounds)
        print(img.shape)
        print(np.isnan(img).sum()/np.prod(img.shape))
        print(np.nanmax(img))
        print(np.nanmin(img))
        print(np.nanmean(img))

land_class_resamp.tif
BoundingBox(left=-116.33326659793262, bottom=43.553469077608824, right=-114.80927471842385, top=44.514037610917825)
(21603, 23940)
0.0
126
20
88.03744724956398
trees_resamp.tif
BoundingBox(left=-116.33326659793262, bottom=43.553469077608824, right=-114.80927471842385, top=44.514037610917825)
(21603, 23940)
0.0009447580128552801
73.0
0.0
23.06897104988965
land_forms_resamp.tif
BoundingBox(left=-116.33326659793262, bottom=43.553469077608824, right=-114.80927471842385, top=44.514037610917825)
(21603, 23940)
0.0
42
11
27.749908381254173
lithography_resamp.tif
BoundingBox(left=-116.33326659793262, bottom=43.553469077608824, right=-114.80927471842385, top=44.514037610917825)
(21603, 23940)
0.0
19
0
5.9095195343819436
